In [10]:
import cv2, time, pathlib, matplotlib.pyplot as plt

# --- stream & output ---
M3U8     = "https://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8"
SNAPFILE = pathlib.Path("tt2_snapshot.jpg")
TIMEOUT  = 15           # seconds to keep trying

print("Opening HLS stream…")
cap  = cv2.VideoCapture(M3U8, cv2.CAP_FFMPEG) 
start = time.time()

while True:
    ok, frame = cap.read()
    if ok:
        SNAPFILE.write_bytes(cv2.imencode(".jpg", frame)[1])
        print("✅  First frame saved to", SNAPFILE.resolve())
        break
    if time.time() - start > TIMEOUT:
        cap.release()
        raise RuntimeError(f"No frame within {TIMEOUT}s; "
                           "check URL, network, or OpenCV-FFmpeg build.")
    time.sleep(0.25)      # small back-off loop

cap.release()

# --- display the frame so you can click & measure ROIs ---
plt.figure(figsize=(10,6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Top Thrill 2 – front-tower camera (first live frame)")


Opening HLS stream…
✅  First frame saved to \\nas01.itap.purdue.edu\myhome\setiawa\puhome Docs\tt2check\tt2_snapshot.jpg


Text(0.5, 1.0, 'Top Thrill 2 – front-tower camera (first live frame)')

In [ ]:
import cv2, matplotlib.pyplot as plt
%matplotlib tk

IMG = "tt2_snapshot.jpg"      # path to your saved frame

# --- load & display ---
img = cv2.imread("tt2_snapshot.jpg")
plt.figure(figsize=(12,7))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Click BL & TR for BOT, MID, TOP; press Enter when done")
coords = plt.ginput(6)
plt.close()

# --- convert to x, y, w, h ---
rois = []
for i in range(0, 6, 2):
    (x0, y0), (x1, y1) = coords[i], coords[i+1]
    x, y   = int(min(x0, x1)), int(min(y0, y1))
    w, h   = int(abs(x1 - x0)), int(abs(y1 - y0))
    rois.append((x, y, w, h))

print(f"ROI_BOT = {rois[0]}\nROI_MID = {rois[1]}\nROI_TOP = {rois[2]}")


ROI_BOT = (635, 695, 39, 70)
ROI_MID = (666, 445, 24, 117)
ROI_TOP = (501, 501, 22, 86)


In [13]:
import cv2, streamlink, itertools, re, time

M3U8_PAGE = "https://pixelcaster.com/live/cedarpoint/https/cam2.html"

HOSTS = [
    "https://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "https://cs3.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "https://cs2.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs3.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs2.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
]

def open_stream():
    """Try known hosts, then let Streamlink resolve the current one."""
    for url in HOSTS:
        cap = cv2.VideoCapture(url, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print(f"[stream] connected → {url}")
            return cap
        cap.release()

    # — Streamlink fallback —
    try:
        hls = streamlink.streams(M3U8_PAGE)["best"].url
        cap  = cv2.VideoCapture(hls, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print(f"[stream] resolved via Streamlink → {hls}")
            return cap
        cap.release()
    except Exception as e:
        print("[stream] Streamlink failed:", e)

    raise RuntimeError("All host attempts and Streamlink resolution failed")


In [14]:
import cv2, time, enum
from collections import deque

# ───────── stream & ROIs ─────────
M3U8_PAGE = "https://pixelcaster.com/live/cedarpoint/https/cam2.html"

HOSTS = [
    "https://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "https://cs3.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "https://cs2.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs3.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
    "http://cs2.pixelcaster.com/live/cedar2.stream/playlist.m3u8",
]

ROI_BOT = (635, 695, 39, 70)
ROI_MID = (666, 445, 24, 117)
ROI_TOP = (501, 501, 22, 86)

def open_stream():
    """Try known hosts, then let Streamlink resolve the current one."""
    for url in HOSTS:
        cap = cv2.VideoCapture(url, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print(f"[stream] connected → {url}")
            return cap
        cap.release()

    # — Streamlink fallback —
    try:
        hls = streamlink.streams(M3U8_PAGE)["best"].url
        cap  = cv2.VideoCapture(hls, cv2.CAP_FFMPEG)
        if cap.isOpened():
            print(f"[stream] resolved via Streamlink → {hls}")
            return cap
        cap.release()
    except Exception as e:
        print("[stream] Streamlink failed:", e)

    raise RuntimeError("All host attempts and Streamlink resolution failed")

cap = open_stream()

# ───────── finite states ─────────
class S(enum.Enum):
    IDLE=0; ASC1=1; RBACK=2; WAIT=3; ASC3=4
state = S.IDLE

# history for velocity smoothing
hist = deque(maxlen=3)         # store last 3 cy
bg   = None                    # adaptive background for ROI_BOT
t0   = None                    # WAIT start time

# thresholds
MOTION_ENTER = 800
MOTION_EXIT  = 350
UP          = -3               # px/frame (negative = up)
DOWN        =  3
WAIT_TIMEOUT = 15              # seconds

# colour helper for console prints
C = {S.IDLE:"\033[37m",S.ASC1:"\033[32m",S.RBACK:"\033[33m",
     S.WAIT:"\033[36m",S.ASC3:"\033[35m", "END":"\033[0m"}

def centroid_from_mask(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    if cv2.contourArea(c) < 40:  # ignore tiny blobs
        return None
    M = cv2.moments(c)
    return (int(M["m01"]/M["m00"])) if M["m00"] else None

while True:
    ok, frame = cap.read()
    if not ok:
        print("\n[stream] lost connection – reconnecting…")
        time.sleep(1)
        cap.release()
        cap = open_stream()
        continue

    x,y,w,h = ROI_BOT
    bot = frame[y:y+h, x:x+w]

    # bg init & update
    if bg is None:
        bg = bot.astype("float32"); continue
    diff   = cv2.absdiff(bot, bg.astype("uint8"))
    motion = (diff > 25).sum()
    cv2.accumulateWeighted(bot.astype("float32"), bg, 0.001)

    # centroid of motion
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY)
    cy_local = centroid_from_mask(mask)           # 0..h-1 or None
    cy = y + cy_local if cy_local is not None else None

    # velocity (smoothed)
    hist.append(cy_local if cy_local is not None else hist[-1] if hist else None)
    if len(hist) >= 3 and None not in hist:
        v = (hist[-1] - hist[0]) / 2.0            # avg over 2 frames
    else:
        v = 0

    # ---------- console debug ----------
    print(f"\r{C[state]}{state.name:<5}{C['END']} "
          f"motion={motion:<4}  v={v:+5.1f}  cy={cy}", end="")

    # ---------- FSM ----------
    if state==S.IDLE and motion>MOTION_ENTER and v<UP:
        state=S.ASC1

    elif state==S.ASC1 and v>DOWN and cy_local is not None:
        state=S.RBACK

    elif state==S.RBACK and (motion<MOTION_EXIT or cy_local is None):
        state, t0 = S.WAIT, time.time()

    elif state==S.WAIT:
        # pause the timeout clock while train blob is still present
        if motion>MOTION_EXIT:
            t0 = time.time()
        elif motion>MOTION_ENTER and v<UP:
            state=S.ASC3
        elif time.time()-t0 > WAIT_TIMEOUT:
            print("\nINCOMPLETE"); state=S.IDLE

    elif state==S.ASC3 and cy is not None and ROI_TOP[1]<=cy<=ROI_TOP[1]+ROI_TOP[3]:
        print("\nSUCCESS"); state=S.IDLE

    elif state==S.ASC3 and v>DOWN and cy is not None and cy>ROI_MID[1]:
        print("\nROLLBACK"); state=S.IDLE


[stream] connected → https://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8
WAIT  motion=155   v= +0.0  cy=None
INCOMPLETE
ASC1  motion=85    v= +0.0  cy=None
[stream] lost connection – reconnecting…
[stream] connected → https://cs4.pixelcaster.com/live/cedar2.stream/playlist.m3u8
WAIT  motion=143   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=164   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=142   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=182   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=287   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=157   v= +0.0  cy=None
INCOMPLETE
WAIT  motion=356   v= +0.0  cy=None

KeyboardInterrupt: 